In [1]:
import argparse
import time
import csv
import pickle
import operator
import datetime
import os
from tqdm import tqdm
import pandas as pd

parser = argparse.ArgumentParser()
parser.add_argument('--dataset', default='amex_explorepoi-poi_category')
args,_ = parser.parse_known_args()
print(args)

Namespace(dataset='amex_explorepoi-poi_category')


In [2]:
def example(data):
    Y=[]
    with open(data, 'r') as fn:
        for ix,line in enumerate(fn):
            if ix<=5:
                Y.append(line.strip("\n"))
    
    return Y

In [3]:
dataset_path = '/home/ec2-user/SageMaker/sequence-based-recommendation/s3_file_processing/dataset_files/amex-yoochoose-format/'
#amex log datasets
with open(os.path.join(dataset_path,'amex_logs_poi_clicks.dat'), 'r') as f, open('amex_log_poi_clicks-withHeader.dat', 'w') as fn:
    fn.write('sessionId,timestamp,itemId,category'+'\n')
    for line in f:
        fn.write(line) 
        
example('amex_log_poi_clicks-withHeader.dat')

['sessionId,timestamp,itemId,category',
 '0,2022-03-01T04:21:32Z,20640',
 '0,2022-03-01T04:21:48Z,57633',
 '1,2022-03-01T13:58:34Z,18506',
 '2,2022-03-01T16:43:02Z,0',
 '2,2022-03-01T16:43:04Z,0']

In [4]:
#amex explore_POI datasets       
with open(os.path.join(dataset_path,'AMEX_explorepoi_poi_category_clicks.dat'), 'r') as f, open('AMEX_explorepoi_poi_category_clicks-withHeader.dat', 'w') as fn:
    fn.write('sessionId,timestamp,itemId,category'+'\n')
    for line in f:
        fn.write(line)
        
example('AMEX_explorepoi_poi_category_clicks-withHeader.dat')

['sessionId,timestamp,itemId,category',
 '1,2022-04-30T21:17:16Z,59508,5',
 '1,2022-04-30T21:17:16Z,59508,5',
 '1,2022-04-30T21:21:35Z,32914,5',
 '1,2022-04-30T21:21:35Z,32914,5',
 '1,2022-04-30T21:22:03Z,11859,4']

In [5]:
with open(os.path.join(dataset_path,'AMEX_explorepoi_poi_context_clicks.dat'), 'r') as f, open('AMEX_explorepoi_poi_context_clicks-withHeader.dat', 'w') as fn:
    fn.write('sessionId,timestamp,itemId,category'+'\n')
    for line in f:
        fn.write(line)
        
example('AMEX_explorepoi_poi_context_clicks-withHeader.dat')

['sessionId,timestamp,itemId,category',
 '1,2022-04-30T21:17:16Z,59508,583',
 '1,2022-04-30T21:17:16Z,59508,583',
 '1,2022-04-30T21:21:35Z,32914,583',
 '1,2022-04-30T21:21:35Z,32914,583',
 '1,2022-04-30T21:22:03Z,11859,583']

In [6]:
with open(os.path.join(dataset_path,'AMEX_explorepoi_category_clicks.dat'), 'r') as f, open('AMEX_explorepoi_category_clicks-withHeader.dat', 'w') as fn:
    fn.write('sessionId,timestamp,itemId,category'+'\n')
    for line in f:
        fn.write(line)
example('AMEX_explorepoi_category_clicks-withHeader.dat')

['sessionId,timestamp,itemId,category',
 '1,2022-04-30T21:17:16Z,5,583',
 '1,2022-04-30T21:17:16Z,5,583',
 '1,2022-04-30T21:21:35Z,5,583',
 '1,2022-04-30T21:21:35Z,5,583',
 '1,2022-04-30T21:22:03Z,4,583']

In [7]:
with open(os.path.join(dataset_path,'AMEX_explorepoi_context_clicks.dat'), 'r') as f, open('AMEX_explorepoi_context_clicks-withHeader.dat', 'w') as fn:
    fn.write('sessionId,timestamp,itemId,category'+'\n')
    for line in f:
        fn.write(line)
        
example('AMEX_explorepoi_context_clicks-withHeader.dat')

['sessionId,timestamp,itemId,category',
 '1,2022-04-30T21:17:16Z,583,5',
 '1,2022-04-30T21:17:16Z,583,5',
 '1,2022-04-30T21:21:35Z,583,5',
 '1,2022-04-30T21:21:35Z,583,5',
 '1,2022-04-30T21:22:03Z,583,4']

In [8]:
if args.dataset == 'diginetica':
    dataset = 'train-item-views.csv'
elif args.dataset =='yoochoose':
    dataset = 'yoochoose-clicks-withHeader.dat'
# elif args.dataset =='amex_explorepoi':
#     dataset = 'AMEX_yoochoose_clicks-withHeader.dat'
# elif args.dataset =='amex_explorepoi_category':
#     dataset = 'AMEX_explorepoi_category_clicks-withHeader.dat'
# elif args.dataset =='amex_explorepoi_context':
#     dataset = 'AMEX_explorepoi_context_clicks-withHeader.dat'
# elif args.dataset =='amex_poi':
#     print('in amex poi')
#     dataset = 'amex_output-withHeader.dat'
# elif args.dataset =='amex_category':
#     dataset = 'amex_category_output-withHeader.dat'
# elif args.dataset =='amex_context':
#     dataset = 'amex_context_output-withHeader.dat'
# else:
#     print('invalid dataset name')
elif args.dataset =='amex_explorepoi-poi_category':
    dataset = 'AMEX_explorepoi_poi_category_clicks-withHeader.dat'
elif args.dataset =='amex_explorepoi-poi_context':
    dataset = 'AMEX_explorepoi_poi_context_clicks-withHeader.dat'
elif args.dataset =='amex_explorepoi-category':
    dataset = 'AMEX_explorepoi_category_clicks-withHeader.dat'
elif args.dataset =='amex_explorepoi-context':
    dataset = 'AMEX_explorepoi_context_clicks-withHeader.dat'
# elif args.dataset =='amex_poi':
#     dataset = 'amex_output-withHeader.dat'
# elif args.dataset =='amex_category':
#     dataset = 'amex_category_output-withHeader.dat'
# elif args.dataset =='amex_context':
#     dataset = 'amex_context_output-withHeader.dat'
    
elif args.dataset =='amex_log-poi':
    dataset = 'amex_log_poi_clicks-withHeader.dat'
elif args.dataset =='amex_log-category':
    dataset = 'amex_log_category_clicks-withHeader.dat'
elif args.dataset =='amex_log-context':
    dataset = 'amex_log_context_clicks-withHeader.dat'
else:
    print('invalid dataset name')

In [9]:
example('AMEX_explorepoi_poi_category_clicks-withHeader.dat')

['sessionId,timestamp,itemId,category',
 '1,2022-04-30T21:17:16Z,59508,5',
 '1,2022-04-30T21:17:16Z,59508,5',
 '1,2022-04-30T21:21:35Z,32914,5',
 '1,2022-04-30T21:21:35Z,32914,5',
 '1,2022-04-30T21:22:03Z,11859,4']

In [10]:
preprocess_output_log = dict({'dataset name':  args.dataset})

amex_sets = set(['amex_explorepoi-poi_category', 'amex_explorepoi-poi_context', 'amex_explorepoi-category', 'amex_explorepoi-context', 'amex_log-poi', 'amex_log-category', 'amex_log-context'])
print("-- Starting @ %ss" % datetime.datetime.now())
with open(dataset, "r") as f:
    if args.dataset == 'yoochoose' or args.dataset in amex_sets:
        reader = csv.DictReader(f, delimiter=',')
    else:
        reader = csv.DictReader(f, delimiter=';')
    sess_clicks = {}
    sess_date = {}
    ctr = 0
    curid = -1
    curdate = None
    for data in tqdm(reader):
        sessid = data['sessionId']
        if curdate and not curid == sessid:
            date = ''
            if args.dataset == 'yoochoose' or args.dataset in amex_sets:
                date = time.mktime(time.strptime(curdate[:19], '%Y-%m-%dT%H:%M:%S'))
#             elif args.dataset in amex_sets: 
#                 date = curdate
            else:
                date = time.mktime(time.strptime(curdate, '%Y-%m-%d'))
            sess_date[curid] = date
        curid = sessid
        if args.dataset == 'yoochoose' or args.dataset in amex_sets:
            item = data['itemId']
        else:
            item = data['itemId'], int(data['timeframe'])
        curdate = ''
        if args.dataset == 'yoochoose' or args.dataset in amex_sets:
            curdate = data['timestamp']
        else:
            curdate = data['eventdate']

        if sessid in sess_clicks:
            sess_clicks[sessid] += [item]
        else:
            sess_clicks[sessid] = [item]
        ctr += 1
        
    date = ''
    if args.dataset == 'yoochoose' or args.dataset in amex_sets:
        date = time.mktime(time.strptime(curdate[:19], '%Y-%m-%dT%H:%M:%S'))
    else:
        date = time.mktime(time.strptime(curdate, '%Y-%m-%d'))
        for i in list(sess_clicks):
            sorted_clicks = sorted(sess_clicks[i], key=operator.itemgetter(1))
            sess_clicks[i] = [c[0] for c in sorted_clicks]
    sess_date[curid] = date
    
print("-- Reading data @ %ss" % datetime.datetime.now())
preprocess_output_log['start time'] = datetime.datetime.now()

-- Starting @ 2022-10-17 17:00:11.085236s


14768it [00:00, 233634.76it/s]

-- Reading data @ 2022-10-17 17:00:11.153031s


In [11]:
len(sess_clicks)

1649

In [12]:
len1_sessions_filtered_count = 0 
# Filter out length 1 sessions
for s in list(sess_clicks):
    if len(sess_clicks[s]) == 1:
        len1_sessions_filtered_count += 1
        del sess_clicks[s]
        del sess_date[s]
        
preprocess_output_log['length 1 sessions filtered'] = len1_sessions_filtered_count

In [13]:
preprocess_output_log

{'dataset name': 'amex_explorepoi-poi_category',
 'start time': datetime.datetime(2022, 10, 17, 17, 0, 11, 153085),
 'length 1 sessions filtered': 87}

In [14]:
len(sess_clicks)

1562

In [15]:
# Count number of times each item appears
iid_counts = {}
for s in sess_clicks:
    seq = sess_clicks[s]
    for iid in seq:
        if iid in iid_counts:
            iid_counts[iid] += 1
        else:
            iid_counts[iid] = 1

sorted_counts = sorted(iid_counts.items(), key=operator.itemgetter(1))

In [16]:
sorted_counts[-5:]

[('12966', 28), ('56712', 29), ('42389', 30), ('32357', 33), ('9889', 57)]

In [17]:
length = len(sess_clicks)
for s in list(sess_clicks):
    curseq = sess_clicks[s]
    filseq = list(filter(lambda i: iid_counts[i] >= 5, curseq))
    if len(filseq) < 2:
        del sess_clicks[s]
        del sess_date[s]
    else:
        sess_clicks[s] = filseq
        
print(f"before filtering:\t {length}")
print(f"after filtering:\t {len(sess_clicks)}")

before filtering:	 1562
after filtering:	 715


In [18]:
# Split out test set based on dates
dates = list(sess_date.items())
maxdate = dates[0][1]

for _, date in dates:
    if maxdate < date:
        maxdate = date
preprocess_output_log['max date'] = maxdate
preprocess_output_log

{'dataset name': 'amex_explorepoi-poi_category',
 'start time': datetime.datetime(2022, 10, 17, 17, 0, 11, 153085),
 'length 1 sessions filtered': 87,
 'max date': 1654055911.0}

In [19]:
amex_splitdate = 6
preprocess_output_log['amex split days before maxdate'] = amex_splitdate
# 7 days for test
splitdate = 0
if args.dataset == 'yoochoose':
    splitdate = maxdate - 86400 * 1  #the number of seconds for a day：86400
elif args.dataset in amex_sets: 
    splitdate = maxdate - 86400 * amex_splitdate #chosen to be 6 days?
else:
    splitdate = maxdate - 86400 * 7 

In [20]:
print('Splitting date', splitdate)      # Yoochoose: ('Split date', 1411930799.0)
tra_sess = filter(lambda x: x[1] < splitdate, dates)
tes_sess = filter(lambda x: x[1] > splitdate, dates)

Splitting date 1653537511.0


In [21]:
# Sort sessions by date
tra_sess = sorted(tra_sess, key=operator.itemgetter(1))     # [(sessionId, timestamp), (), ]
tes_sess = sorted(tes_sess, key=operator.itemgetter(1))     # [(sessionId, timestamp), (), ]
print(len(tra_sess))    # 186670    # 7966257    explore_poi 771 explorepoi_con 1170
print(len(tes_sess))    # 15979     # 15324      explore_poi 84  explorepoi_con 148
print(tra_sess[:3])
print(tes_sess[:3])
print("-- Splitting train set and test set @ %ss" % datetime.datetime.now())

644
71
[('1071', 1646251192.0), ('1082', 1646263575.0), ('1640', 1646274370.0)]
[('624', 1653544398.0), ('1284', 1653574630.0), ('392', 1653584603.0)]
-- Splitting train set and test set @ 2022-10-17 17:00:11.221096s


In [22]:
preprocess_output_log['len training sessions (tra_sess)'] = len(tra_sess)
preprocess_output_log['len test sessions (tes_sess)'] = len(tes_sess)

In [23]:
# Choosing item count >=5 gives approximately the same number of items as reported in paper
item_dict = {}
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_ids = []
    train_seqs = []
    train_dates = []
    item_ctr = 1
    for s, date in tra_sess:
        seq = sess_clicks[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_ids += [s]
        train_dates += [date]
        train_seqs += [outseq]
    print('item count: ')
    preprocess_output_log['item count:'] = item_ctr
    print(item_ctr)     # 43098, 37484, 
                        #explore_poi: 551 explorepoi_category: 4, explorepoi_context: 183
                        #amex: 1164  amexcategory: 4  amexcontext: 482
    return train_ids, train_dates, train_seqs


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_ids = []
    test_seqs = []
    test_dates = []
    for s, date in tes_sess:
        seq = sess_clicks[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < 2:
            continue
        test_ids += [s]
        test_dates += [date]
        test_seqs += [outseq]
    return test_ids, test_dates, test_seqs


tra_ids, tra_dates, tra_seqs = obtian_tra()
tes_ids, tes_dates, tes_seqs = obtian_tes()


item count: 
556


In [24]:
def process_seqs(iseqs, idates):
    out_seqs = []
    out_dates = []
    labs = []
    ids = []
    for id, seq, date in zip(range(len(iseqs)), iseqs, idates):
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
            out_dates += [date]
            ids += [id]
    return out_seqs, out_dates, labs, ids


tr_seqs, tr_dates, tr_labs, tr_ids = process_seqs(tra_seqs, tra_dates)
te_seqs, te_dates, te_labs, te_ids = process_seqs(tes_seqs, tes_dates)
tra = (tr_seqs, tr_labs)
tes = (te_seqs, te_labs)
print(len(tr_seqs))  # explore_poi: 3361 explore_cont: 9998
print(len(te_seqs))  # explore_poi: 298  explore_cont: 1432
print(tr_seqs[:3], tr_dates[:3], tr_labs[:3])
print(te_seqs[:3], te_dates[:3], te_labs[:3])
all = 0

preprocess_output_log['len training seqs (tr_seqs)'] = len(tr_seqs)
preprocess_output_log['len test seqs (te_seqs)'] = len(tr_seqs)

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('avg length: ', all/(len(tra_seqs) + len(tes_seqs) * 1.0))

preprocess_output_log['avg length'] = all/(len(tra_seqs) + len(tes_seqs) * 1.0)
preprocess_output_log

3536
305
[[1], [2], [3, 3, 4, 4]] [1646251192.0, 1646263575.0, 1646274370.0] [1, 2, 4]
[[555, 50, 50, 555], [555, 50, 50], [555, 50]] [1653544398.0, 1653544398.0, 1653544398.0] [555, 555, 50]
avg length:  6.4405099150141645


{'dataset name': 'amex_explorepoi-poi_category',
 'start time': datetime.datetime(2022, 10, 17, 17, 0, 11, 153085),
 'length 1 sessions filtered': 87,
 'max date': 1654055911.0,
 'amex split days before maxdate': 6,
 'len training sessions (tra_sess)': 644,
 'len test sessions (tes_sess)': 71,
 'item count:': 556,
 'len training seqs (tr_seqs)': 3536,
 'len test seqs (te_seqs)': 3536,
 'avg length': 6.4405099150141645}

In [25]:
preprocess_output_log = {k:[v] for k,v in preprocess_output_log.items()}
print(preprocess_output_log)

{'dataset name': ['amex_explorepoi-poi_category'], 'start time': [datetime.datetime(2022, 10, 17, 17, 0, 11, 153085)], 'length 1 sessions filtered': [87], 'max date': [1654055911.0], 'amex split days before maxdate': [6], 'len training sessions (tra_sess)': [644], 'len test sessions (tes_sess)': [71], 'item count:': [556], 'len training seqs (tr_seqs)': [3536], 'len test seqs (te_seqs)': [3536], 'avg length': [6.4405099150141645]}


In [26]:
output_df = pd.DataFrame.from_dict(preprocess_output_log) 
output_df

,dataset name,start time,length 1 sessions filtered,max date,amex split days before maxdate,len training sessions (tra_sess),len test sessions (tes_sess),item count:,len training seqs (tr_seqs),len test seqs (te_seqs),avg length
0,amex_explorepoi-poi_category,2022-10-17 17:00:11.153085,87,1.654056e+09,6,644,71,556,3536,3536,6.44051


In [27]:
if args.dataset in amex_sets:
    if not os.path.exists(args.dataset):
        os.makedirs(args.dataset)
    pickle.dump(tra, open(args.dataset + '/train.txt', 'wb'))
    pickle.dump(tes, open(args.dataset + '/test.txt', 'wb'))
    pickle.dump(tra_seqs, open(args.dataset + '/all_train_seq.txt', 'wb'))

In [28]:
dataset_path = '/home/ec2-user/SageMaker/sequence-based-recommendation/NARM/datasets/run_dataset/'+args.dataset
dataset_path

'/home/ec2-user/SageMaker/sequence-based-recommendation/NARM/datasets/run_dataset/amex_explorepoi-poi_category'

In [41]:
with open(os.path.join(dataset_path,"test.txt"),"rb") as f:
    test_set=pickle.load(f)

for idx,(x,y) in enumerate(zip(test_set[0],test_set[1])):
    if idx<=5:
        print(x,y)

[555, 50, 50, 555] 555
[555, 50, 50] 555
[555, 50] 50
[555] 50
[66] 66
[480, 480, 265] 265


In [44]:
print(test_set[0][0], test_set[1][0])
print(test_set[0][1], test_set[1][1])
print(test_set[0][2], test_set[1][2])
print(test_set[0][3], test_set[1][3])
print(test_set[0][4], test_set[1][4])
print(test_set[0][5], test_set[1][5])

[555, 50, 50, 555] 555
[555, 50, 50] 555
[555, 50] 50
[555] 50
[66] 66
[480, 480, 265] 265
